In [2]:
from hera.shared import global_config
from hera.auth import ArgoCLITokenGenerator

global_config.host = "https://localhost:2746"
global_config.token = ArgoCLITokenGenerator
global_config.verify_ssl = False

## Hera - compiles workflow template manifest (i.e. define `Flow`)

WorkFlowTemplates crucially allow the definition of parameters as input variables to the entire workflow *without specifying a value*. You can also specify the default value, but it is possible to omit in cases where a sensible value just doesnt apply.

Submitting these WorkFlow templates doesnt execute anything on the ArgoWorkflow engine, but rather adds a template that future `Workflow`s can be derived from, together with configurable workflow input parameters.

This allows for the following entity mapping between bettmensch.ai pipelines and ArgoWorkflow resources:

| bettmensch.ai | ArgoWorkflow      |
| --------------|-------------------|
| `Flow`        | `WorkflowTemplate`|
| `Run`         | `Workflow`        |

In [4]:
from hera.workflows import Workflow, WorkflowTemplate, DAG, script, Parameter, models as m

@script(outputs=[Parameter(name="coin",value_from=m.ValueFrom(path="./coin_output.txt"))])
def set_coin(coin: str):
    # import random
    # res = "heads" if random.randint(0, 1) == 0 else "tails"
    with open('./coin_output.txt','w') as output:
        output.write(coin)
        
@script()
def show_coin(coin: str):
    print(f"it was {coin}")
    
with WorkflowTemplate(
    generate_name="set-a-coin-", 
    entrypoint="Coin-set",
    namespace="argo",
    arguments=[Parameter(name="coin")],
    # the Workflow referencing this template inherits the `spec` level `workflow_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Workflow CRD manifest before being applied to the K8s cluster
    workflow_metadata={
        'annotations':{'annotation_key_a':'annotation_value_a'},
        'labels':{'label_key_a':'label_value_a'}
    },
    # the Workflow referencing this template inherits the `spec` level `pod_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Pod manifest(s) before being applied to the K8s cluster.
    # for whatever reason, the (apparently?) pod level manifests shown on the ArgoWorkflow server dashboard do not show this
    pod_metadata={ 
        'annotations':{'annotation_key_b':'annotation_value_b'},
        'labels':{'label_key_b':'label_value_b'}
    }) as wt:
    with DAG(name="Coin-set"):
        set_coin_task = set_coin(name="Set-a-coin",arguments=[wt.get_parameter('coin').as_argument()])
        
        show_coin_task = show_coin(name="Show-a-coin",arguments=[set_coin_task.get_parameter('coin').as_argument()],depends='Set-a-coin') # accesses the stdout from the fc task and checks if its "heads"

In [5]:
import yaml

with open('hera-template-3.yaml','w') as pipeline_file:
    yaml.dump(wt.to_yaml(),pipeline_file)

# Hera - submit Workflow template (i.e. submit `Flow`)

In [6]:
wt.create()

ConnectionError: HTTPSConnectionPool(host='localhost', port=2746): Max retries exceeded with url: /api/v1/workflow-templates/argo (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000000BC82C061D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

# Hera - generate workflow from template (i.e. configure `Run`)

In [7]:

from hera.workflows import Workflow
from hera.workflows.models import WorkflowTemplateRef

# # using the workflow template reference
# wt_ref = WorkflowTemplateRef(name="set-a-coin-rl4sj")

# w1 = Workflow(
#     generate_name="set-a-coin-from-template-ref-",
#     workflow_template_ref=wt_ref,
#     namespace="argo",
#     arguments=Parameter(name="coin",value='new coin')
# )

# copy pasting entrypoint and templates from workflow template
w2 = Workflow(
    generate_name="set-a-coin-from-template-copy-",
    templates=wt.templates,
    entrypoint=wt.entrypoint,
    # when submitting via .create(), the workflow metadata does get merged into the Workflow CRD manifest before being applied to the K8s cluster
    workflow_metadata={
        'annotations':{'annotation_key_1':'annotation_value_1'},
        'labels':{'label_key_1':'label_value_1'}
    },
    # when submitting via .create(), the workflow metadata does get merged into the Pod manifest(s) before being applied to the K8s cluster.
    # for whatever reason, the (apparently?) pod level manifests shown on the ArgoWorkflow server dashboard do not show this
    pod_metadata={ 
        'annotations':{'annotation_key_2':'annotation_value_2'},
        'labels':{'label_key_2':'label_value_2'}
    },
    namespace="argo",
    arguments=Parameter(name="coin",value='new coin')
)

# Hera - submit workflow (i.e. execute `Run`)

In [60]:
#w1.create()
w2.create()

import yaml

# with open('hera-pipeline-3-from-ref.yaml','w') as pipeline_file:
#     yaml.dump(w1.to_yaml(),pipeline_file)

with open('hera-pipeline-3-from-copy.yaml','w') as pipeline_file:
    yaml.dump(w2.to_yaml(),pipeline_file)

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
